In [40]:
import SNPLIB
import math
import numpy as np
import numpy.linalg as npl
import matplotlib.pyplot as plt
%matplotlib notebook

In [41]:
Fst = 0.1
num_generations = 200
effective_sample_size = math.floor(num_generations/2/(1-math.exp(-Fst)))

In [42]:
num_snps = 100000
num_samples = 4000
aaf = np.random.rand(num_snps)
af = SNPLIB.UpdateAf(aaf,2,num_generations,effective_sample_size)

In [43]:
pop = np.zeros((num_samples,2),dtype='double',order='F')
pop[:,0] = np.sort(np.random.beta(0.1,0.1,num_samples))
pop[:,1] = 1.0-pop[:,0]

In [44]:
af = pop@af
obj = SNPLIB.SNPLIB(16)

In [45]:
obj.GenerateIndividuals(af)
scores = obj.CalcSUGIBSScores(1)

In [ ]:
cov = np.ones((num_samples,2),dtype='double',order='F')
cov[:,1] = scores[:,0]
adjgrm = obj.CalcAdjustedGRM(cov)
plt.imshow(adjgrm[0],cmap='gray',vmin=-0.025,vmax=0.025)

In [50]:
scores.shape

(4000, 1)